In [2]:
 # boolean network packages
from sad2_final_project.boolean_bn import BN
from sad2_final_project.boolean_bn import simulate_trajectories_to_csv 
from sad2_final_project.analysis import BooleanNetworkExperiment
from boolean import BooleanAlgebra 
# system packages
import os
from pathlib import Path


# paths
## set global dir
cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())
## create paths 
DATA_PATH = Path('data')
# # TODO OBSOLETE 
# DATASET_PATH = DATA_PATH / 'datasets'
# BN_TRUE_PATH = DATA_PATH / 'bn_ground_truth'
# BN_DATASET_BNFINDER_FORMAT_PATH = DATA_PATH / 'datasets_bnfinder_format'
# BN_RESULT_PATH = DATA_PATH / "result"
## create directories
!mkdir {DATA_PATH}
# TODO OBSOLETE
# !mkdir -p {DATASET_PATH}
# !mkdir -p {BN_TRUE_PATH}
# !mkdir -p {BN_DATASET_BNFINDER_FORMAT_PATH}
# !mkdir -p {BN_RESULT_PATH}




/home/maxi7524/repositories/SAD2_final_project
mkdir: cannot create directory ‘data’: File exists


## Part 1 

### 1. Construct boolean networks (way to do it ?)
#### Task
Construct several Boolean networks with sizes (measured by the number of nodes or variables) ranging from 5 to 16.† Each node should have no more than three parent nodes, and the Boolean functions governing individual nodes should be generated at random.

#### Goal
The goal of this study is to determine how the type and amount of time-series data generated from Boolean network dynamics affect the accuracy of dynamic Bayesian network (DBN) structure inference. In particular, we aim to identify:
- how the presence of attractor states in trajectories influences reconstruction accuracy,
- how trajectory length, sampling frequency, and the number of trajectories affects model metrics
- how these effects depend on network size and update dynamics,
- which data-generation techniques yield stable and informative reconstructions. 

Because synchronous and asynchronous Boolean network updates correspond to different classes of stochastic processes, all results are analyzed separately for these two update modes. Furthermore, since different scoring functions (MDL and BDe) are not directly comparable in scale, they are treated as different experiment groups for same boolean networks. 

#### Methodology

##### Relation Between Trajectory Length and Entering Attractors

**Objective.**
To characterize how trajectory length is related to the probability of entering attractors as a function of network size and dynamics.

**Experimental design.**

- The target attractor proportion is **not controlled**; trajectories evolve naturally.
- Trajectory lengths are varied in increments proportional to network size:
    - from 5 steps to 50 by 5
    - from 50 steps to 200 by 10
- Networks are grouped by size (from 4 to 16 nodes, in steps of two).
- The number of parents per node is randomly chosen from set of $\{1,2,3\}$ to avoid conditioning results on a fixed connectivity pattern.

**Measured quantities.**

- Probability of reaching an attractor as a function of trajectory length.
- How different groups (below TODO - inner link) differ in in this probability.

**Rationale.**
Attractor entry is an emergent property of the dynamics. Controlling it directly is undesirable, as it would introduce selection bias. This experiment instead characterizes the **natural scaling behavior** of Boolean network dynamics.

---

##### Impact of Sampling Frequency

**Objective.**
To determine how temporal subsampling affects autocorrelation, effective sample size, and reconstruction accuracy.

Dynamic Bayesian network inference assumes conditional independence of observations given parent states in the previous time slice. Excessive temporal dependence violates this assumption in practice by introducing redundant observations.

**Experimental design.**

- For fixed networks and trajectory lengths, datasets are generated using multiple sampling frequencies (1, 2, 3, 4, 5).
- For each dataset
    - ACF and ESS are computed,
    - MDL and BDe scores are extracted from BNFinder2 logs,

**Analysis.**
Accuracy is analyzed jointly as a function of: 
<!-- What does it mean jointly ?? -->

* sampling frequency,
* ESS,
* scoring function (MDL or BDe).

**Rationale.**
This experiment identifies sampling regimes that balance reduced autocorrelation against loss of dynamic information due to over-subsampling.

---

##### Amount of Trajectories Required for Stable Inference

**Objective.**
To determine how many independent trajectories are required to obtain statistically stable reconstructions.

**Experimental design.**

- Sampling frequency and trajectory length are fixed to values identified as near-optimal in previous experiments.
- The number of trajectories per dataset is gradually increased - from 10 to 100 by 10.
- For each setting, multiple (30) independent repetitions are performed to obtain convergent distribution .

**Evaluation.**

- Reconstruction accuracy is summarized using distributions (score functions).
- Stability is assessed by observing convergence of accuracy metrics as the number of trajectories increases.
- No classical parametric hypothesis test is assumed; instead, convergence trends is reported.
<!-- Nie znalazłem żadnego sensownego -->

**Rationale.**
Due to the randomness of Boolean functions and initial states, averaging over multiple networks is necessary to separate systematic effects from instance-specific variability.

---

#### Sets

To ensure reproducibility and comparisons, experiments are organized into predefined sets.
##### Groups
1. **Network size groups**
   Networks are grouped by number of nodes (5–16) to analyze scaling behavior.
2. **Update mode groups**
   Synchronous and asynchronous updates are treated separately, as they correspond to different stochastic processes (deterministic map vs stochastic transition system).
3. **Scoring function groups**
   MDL and BDe are analyzed independently. Absolute score values are not compared across scoring functions; only trends with respect to accuracy are considered.
4. **Random function sets**
   For each experimental condition, multiple independently generated Boolean networks are used.
   Random seeds are fixed per network/experiment (TODO ?) instance so that different data-generation parameters (sampling frequency, trajectory length, number of trajectories) are evaluated on identical underlying networks.
   <!-- is not possible we just need to use seed to experiments are reproducle -->

---

##### Averaging and Distributions

All reported results are based on **distributions**, not single values.
Depending on the experiment, averaging is performed over:

- trajectories (within a dataset),
- independent datasets,
- independently generated networks.

The aggregation strategy is explicitly chosen for each experiment to match the source of variability under investigation.


<!-- ### Impact of Proportion of Attractor States in Trajectories

**Objective.**
To quantify how an increasing proportion of attractor states in trajectories affects the sensitivity of network structure reconstruction.

The central hypothesis is that a high proportion of attractor states leads to strong temporal autocorrelation, which **reduces the effective number of independent observations**, thereby decreasing the sensitivity (recall) of detected edges in the inferred network.

**Experimental design.**

- For each network, datasets are generated with controlled proportions of attractor states:
  (0%, 10%, \ldots, 100%).
- Trajectories are made sufficiently long to ensure that attractors are reached whenever the target proportion is nonzero.
- Sampling frequency is varied to contrast regimes of strong versus weak temporal dependence.

**Autocorrelation analysis.**
For each dataset, temporal dependence is quantified using the **autocorrelation function (ACF)**:
[
\rho(k) = \frac{\mathrm{Cov}(X_t, X_{t+k})}{\mathrm{Var}(X_t)}
]
computed separately for each node and then aggregated (mean or maximum across nodes).

From the ACF, the **effective sample size (ESS)** is estimated:
[
\mathrm{ESS} \approx \frac{N}{1 + 2\sum_{k=1}^{K} \rho(k)},
]
where (N) is the nominal number of observations and (K) is the truncation lag where autocorrelation becomes negligible.

Reconstruction accuracy is then analyzed as a function of ESS rather than nominal sample size.

**Rationale.**
This isolates the effect of attractor-induced redundancy and avoids conflating “amount of data” with “amount of information”. -->




In [3]:
bn_random = BN(num_nodes=3, mode="asynchronous", trajectory_length=50, n_parents_per_node=[1])

2026-01-21 00:15:46,342 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 3 nodes in asynchronous mode.
2026-01-21 00:15:46,345 [INFO] sad2_final_project.boolean_bn.bn: Generating random Boolean functions.
2026-01-21 00:15:46,346 [INFO] sad2_final_project.boolean_bn.bn: Generated 3 Boolean functions.
2026-01-21 00:15:46,348 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-21 00:15:46,349 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-21 00:15:46,350 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-21 00:15:46,352 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 8 states and 22 transitions.
2026-01-21 00:15:46,355 [INFO] sad2_final_project.boolean_bn.bn: Found 1 attractors.


[{(1, 0, 1), (1, 1, 0), (0, 1, 0), (0, 0, 0), (1, 0, 0), (0, 0, 1), (1, 1, 1), (0, 1, 1)}]


In [7]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    # paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',
    # experiment values
    num_nodes=[4, 6, 8],
    update_mode=["synchronous", "asynchronous"],
    trajectory_length=list(range(5, 21, 5)),
    n_trajectories=[100],
    sampling_frequency=[1],
    score_functions=["MDL", "BDe"],
    n_parents_per_node=[[1, 2, 3, 4]],
    # amount of samples per experiment 
    n_repetitions=1,
    simulate_trajectories_to_csv_kwargs = {
            # "sampling_frequency": 1,
            "target_attractor_ratio": 0.4,  # Approximate fraction of trajectory in attractor (0-1)
            "tolerance": 0.3,               # Allowed deviation from the calculated entrance step (0-1)
            "max_iter": 50,                 # Maximum attempts to generate a valid state per step before restarting
            "max_trajectory_restarts": 10_000  # Maximum number of trajectory restarts allowed
        }
) 
## case: Impact of sampling frequency 
# exp_sampling_frequency
## case: Amount of Trajectories 
# exp_trajectory_amount

In [8]:
exp_trajectory_length.show_experiment_df()

,num_nodes,update_mode,trajectory_length,n_trajectories,sampling_frequency,score_function,n_parents_per_node,rep_id,condition_id
0,4,synchronous,5,100,1,MDL,"[1, 2, 3, 4]",0,0000
1,4,synchronous,5,100,1,BDe,"[1, 2, 3, 4]",0,0001
2,4,synchronous,10,100,1,MDL,"[1, 2, 3, 4]",0,0002
3,4,synchronous,10,100,1,BDe,"[1, 2, 3, 4]",0,0003
4,4,synchronous,15,100,1,MDL,"[1, 2, 3, 4]",0,0004
5,4,synchronous,15,100,1,BDe,"[1, 2, 3, 4]",0,0005
6,4,synchronous,20,100,1,MDL,"[1, 2, 3, 4]",0,0006
7,4,synchronous,20,100,1,BDe,"[1, 2, 3, 4]",0,0007
8,4,asynchronous,5,100,1,MDL,"[1, 2, 3, 4]",0,0008
9,4,asynchronous,5,100,1,BDe,"[1, 2, 3, 4]",0,0009


In [8]:
row

num_nodes                        3
update_mode            synchronous
trajectory_length                5
n_trajectories                 100
sampling_frequency               1
score_function                 MDL
n_parents_per_node    [1, 2, 3, 4]
rep_id                           0
condition_id                  0000
Name: 0, dtype: object

In [9]:
# exp_trajectory_length._run_single_condition()
# _, row = exp_trajectory_length.show_experiment_df().iterrows().__next__()
exp_trajectory_length.run_experiment()

2026-01-21 00:17:05,730 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 4 nodes in synchronous mode.
2026-01-21 00:17:05,732 [INFO] sad2_final_project.boolean_bn.bn: Generating random Boolean functions.
2026-01-21 00:17:05,733 [INFO] sad2_final_project.boolean_bn.bn: Generated 4 Boolean functions.
2026-01-21 00:17:05,734 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-21 00:17:05,735 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-21 00:17:05,737 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-21 00:17:05,741 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 16 states and 16 transitions.
2026-01-21 00:17:05,742 [INFO] sad2_final_project.boolean_bn.bn: Found 1 attractors.
2026-01-21 00:17:05,743 [INFO] sad2_final_project.boolean_bn.bn: Saving ground truth edges to: data/trajectory_length_vs_attractors/bn_ground_truth/0000.c

[{(0, 1, 0, 1), (1, 0, 1, 0)}]


2026-01-21 00:17:05,958 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (3/10000)
2026-01-21 00:17:05,963 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (5/10000)
2026-01-21 00:17:05,979 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (7/10000)
2026-01-21 00:17:05,984 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (9/10000)
2026-01-21 00:17:05,989 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (11/10000)
2026-01-21 00:17:06,001 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 5, Attractors: 3, Transients: 1
2026-01-21 00:17:06,002 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.30, max_iter 50
2026-01-21 00:17:06,003 [INFO] sad2_final_project.boolean_

Done.
-> Loading external data from: data/trajectory_length_vs_attractors/datasets/0000.csv
   Loaded dataset with shape: (500, 4)
-> Data written to data/trajectory_length_vs_attractors/datasets_bnfinder/0000.txt
-> Loading ground truth from: data/trajectory_length_vs_attractors/bn_ground_truth/0000.csv

=== STARTING INFERENCE ===
-> Running BNFinder with score: MDL...


2026-01-21 00:17:07,518 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 4 nodes in synchronous mode.
2026-01-21 00:17:07,519 [INFO] sad2_final_project.boolean_bn.bn: Generating random Boolean functions.
2026-01-21 00:17:07,520 [INFO] sad2_final_project.boolean_bn.bn: Generated 4 Boolean functions.
2026-01-21 00:17:07,520 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-21 00:17:07,521 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-21 00:17:07,522 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-21 00:17:07,524 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 16 states and 16 transitions.
2026-01-21 00:17:07,525 [INFO] sad2_final_project.boolean_bn.bn: Found 3 attractors.
2026-01-21 00:17:07,526 [INFO] sad2_final_project.boolean_bn.bn: Saving ground truth edges to: data/trajectory_length_vs_attractors/bn_ground_truth/0001.c

...

Done.
-> Loading external data from: data/trajectory_length_vs_attractors/datasets/0001.csv
   Loaded dataset with shape: (500, 4)
-> Data written to data/trajectory_length_vs_attractors/datasets_bnfinder/0001.txt
-> Loading ground truth from: data/trajectory_length_vs_attractors/bn_ground_truth/0001.csv

=== STARTING INFERENCE ===
-> Running BNFinder with score: BDe...


2026-01-21 00:17:14,352 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 4 nodes in synchronous mode.
2026-01-21 00:17:14,353 [INFO] sad2_final_project.boolean_bn.bn: Generating random Boolean functions.
2026-01-21 00:17:14,354 [INFO] sad2_final_project.boolean_bn.bn: Generated 4 Boolean functions.
2026-01-21 00:17:14,355 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-21 00:17:14,356 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-21 00:17:14,357 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-21 00:17:14,359 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 16 states and 16 transitions.
2026-01-21 00:17:14,360 [INFO] sad2_final_project.boolean_bn.bn: Found 1 attractors.
2026-01-21 00:17:14,360 [INFO] sad2_final_project.boolean_bn.bn: Saving ground truth edges to: data/trajectory_length_vs_attractors/bn_ground_truth/0002.c

   BNFinder execution failed!
   Error Output:
 None
[BDe] Failed: Command '['python2', '/home/maxi7524/.local/bin/bnf', '-e', PosixPath('data/trajectory_length_vs_attractors/datasets_bnfinder/0001.txt'), '-n', PosixPath('data/trajectory_length_vs_attractors/results/0001_BDe.sif'), '-s', 'BDe', '-v']' returned non-zero exit status 1.

=== DONE ===
[{(0, 0, 1, 0), (1, 1, 0, 1), (1, 0, 1, 0)}]


2026-01-21 00:17:14,559 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (81/10000)
2026-01-21 00:17:14,563 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (83/10000)
2026-01-21 00:17:14,567 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (85/10000)
2026-01-21 00:17:14,572 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (87/10000)
2026-01-21 00:17:14,576 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (89/10000)
2026-01-21 00:17:14,581 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (91/10000)
2026-01-21 00:17:14,590 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (93/10000)
2026-01-21 00:17:14,595 [WARNING] sad2_final_project.boolean_b

RuntimeError: Failed to generate trajectory after 10000 restarts.

In [ ]:
# TODO konstrukcja sieci

## TODO - tworzymy generator który będzie nam zwracał odpowiednie sieci

## TODO - losujemy na zadanych warunkach nasze sieci

## zapisujemy nasze wygenerowane funkcje w klasie bn (można domyślnie) 

## TODO zapisujemy naszą klase jako ground trough - tutaj trzeba zrobic metodę do robienia tego !!!

# Create randomo BN 
bn_random = BN(num_nodes=9, mode="asynchronous", trajectory_length=50, n_parents_per_node=[2, 3])
# save ground truth for 
bn_random.save_ground_truth(BN_TRUE_PATH / 'condition_2.csv')

bn_random.functions

2026-01-20 13:16:01,666 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 9 nodes in asynchronous mode.
2026-01-20 13:16:01,669 [INFO] sad2_final_project.boolean_bn.bn: Generating random Boolean functions.
2026-01-20 13:16:01,671 [INFO] sad2_final_project.boolean_bn.bn: Generated 9 Boolean functions.
2026-01-20 13:16:01,673 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-20 13:16:01,674 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-20 13:16:01,674 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-20 13:16:01,917 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 512 states and 2750 transitions.
2026-01-20 13:16:01,925 [INFO] sad2_final_project.boolean_bn.bn: Found 1 attractors.
2026-01-20 13:16:01,926 [INFO] sad2_final_project.boolean_bn.bn: Saving ground truth edges to: data/test/bn_ground_truth/condition_2.csv
2026-01-20 13

[{(0, 1, 0, 1, 0, 1, 0, 0, 0)}]


[AND(OR(Symbol('x2'), NOT(Symbol('x3'))), Symbol('x6')),
 AND(OR(NOT(Symbol('x0')), Symbol('x3')), NOT(Symbol('x8'))),
 AND(NOT(Symbol('x0')), NOT(Symbol('x3'))),
 OR(OR(NOT(Symbol('x8')), NOT(Symbol('x5'))), Symbol('x2')),
 OR(AND(NOT(Symbol('x7')), NOT(Symbol('x5'))), Symbol('x2')),
 OR(OR(Symbol('x3'), NOT(Symbol('x5'))), NOT(Symbol('x6'))),
 AND(AND(Symbol('x2'), NOT(Symbol('x0'))), NOT(Symbol('x1'))),
 AND(Symbol('x7'), Symbol('x8')),
 AND(OR(NOT(Symbol('x6')), NOT(Symbol('x1'))), NOT(Symbol('x3')))]

### 2. Simulate boolean networks 
#### Task
Simulate trajectories of the generated networks in both synchronous and asynchronous modes to create datasets. The datasets should vary in 
- the proportion of transient and attractor states,
- the trajectory sampling frequency (i.e. the number of time steps between consecutive sampled states), and 
- their overall size (i.e. the number and length of trajectories used to construct each dataset);


In [ ]:
# TODO symulacja sieci

##  Korzystamy z funkcji Michała, do symulacji
# save_trajectories_to_csv(bn_instance=bn, num_trajectories=100, output_file= DATASET_PATH / 'condition_1.csv' )

simulate_trajectories_to_csv(bn_instance=bn_random, num_trajectories=50, output_file= DATASET_PATH / 'condition_2.csv' )

## TODO musimy zrobić dla każdej sieci różne datasety, to jet network_1_condition_x i mamy x warunków dla każdego przypadku, żeby zobaczyć jak się zachowuje znajdywanie takiej sieci w zależności od dataset'u 

2026-01-20 13:16:10,566 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-20 13:16:10,578 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 50, Attractors: 48, Transients: 99
2026-01-20 13:16:10,579 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-20 13:16:10,590 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 50, Attractors: 63, Transients: 84
2026-01-20 13:16:10,591 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-20 13:16:10,603 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 50, Attractors: 65, Transients: 82
2026-01-20 13:16:10,604 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-20 13:

Done.


### 3. Build dynamic Bayesian newtorks BNFinder2
#### Task
Use the datasets to infer dynamic Bayesian networks with the BNFinder2 software tool. Unlike classical Bayesian networks, dynamic Bayesian networks can contain cycles, making them more suitable for this task. Consider two scoring functions (referred to as scoring criteria in the BNFinder2 terminology): Minimal Description Length (MDL) and BDe (Bayesian–Dirichlet equivalence).

In [8]:
# # TODO temp - problem with path
# p = BN_TRUE_PATH / 'condition_1.csv'
# print("cwd:", Path.cwd())
# print("path:", p)
# print("exists:", p.exists())
# print("is_file:", p.is_file())
# print("parent exists:", p.parent.exists())
# print("files in parent:", list(p.parent.iterdir()) if p.parent.exists() else None)
# # TODO temp - problem with path
p = BN_TRUE_PATH / 'condition_2.csv'
print("cwd:", Path.cwd())
print("path:", p)
print("exists:", p.exists())
print("is_file:", p.is_file())
print("parent exists:", p.parent.exists())
print("files in parent:", list(p.parent.iterdir()) if p.parent.exists() else None)


cwd: /home/maxi7524/repositories/SAD2_final_project
path: data/test/bn_ground_truth/condition_1.csv
exists: True
is_file: True
parent exists: True
files in parent: [PosixPath('data/test/bn_ground_truth/condition_2.csv'), PosixPath('data/test/bn_ground_truth/condition_1.csv')]
cwd: /home/maxi7524/repositories/SAD2_final_project
path: data/test/bn_ground_truth/condition_2.csv
exists: True
is_file: True
parent exists: True
files in parent: [PosixPath('data/test/bn_ground_truth/condition_2.csv'), PosixPath('data/test/bn_ground_truth/condition_1.csv')]


In [7]:
# TODO odbudowa sieci 

## TODO bierezmy data sety z poprzedniego ćwiczenia i je tutaj odtwarzamy za pomocą tego skryptu, więc powinniśmy móc go po prostu uruchomić
from sad2_final_project.bnfinder import run_bnfinder

datasets = [DATASET_PATH / 'condition_1.csv', DATASET_PATH / 'condition_2.csv']
datasets = [DATASET_PATH / 'condition_2.csv']
# IDEA - conditions 
for d in datasets:
    model_name = str(d.stem)
    run_bnfinder(
        #TODO -- tutaj trzeba dopasować odpowienio patha'a do próby, żeby bezmyślnei tego nei skopiować|
        dataset_path=d,
        ground_truth_path=BN_TRUE_PATH / (model_name + '.csv'),
        bnf_file_path=BN_DATASET_BNFINDER_FORMAT_PATH / (model_name + '.txt'),
        trained_model_name=BN_RESULT_PATH / model_name,
        metrics_file=BN_RESULT_PATH / (model_name + '_metric.csv'),
    )
## TODO przeliczenie miar (to trzeba osobny skrypt na to, bo ja bym to chciał zestawić potem w jedną tabele)

-> Loading external data from: data/test/datasets/condition_2.csv
   Loaded dataset with shape: (2500, 9)
-> Data written to data/test/datasets_bnfinder_format/condition_2.txt
-> Loading ground truth from: data/test/bn_ground_truth/condition_2.csv

=== STARTING INFERENCE ===
-> Running BNFinder with score: MDL...


/home/maxi7524/repositories/SAD2_final_project/src/sad2_final_project/bnfinder/bnfinder.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges.add((str(row[0]), str(row[1]))) # Parent, Child


   Success! Output saved to data/test/result/condition_2_MDL.sif
[MDL] Inferred 28 edges.
   (Ground truth file found, evaluating metrics)
-> Running BNFinder with score: BDE...
   Success! Output saved to data/test/result/condition_2_BDE.sif
[BDE] Inferred 39 edges.
   (Ground truth file found, evaluating metrics)

=== DONE ===


### 4. Asses Quaility 
#### Task
Evaluate the accuracy of the reconstructed network structures with respect to the characteristics of the datasets and the scoring functions used. Consider the original Boolean network (i.e. the one used to generate the dataset) as the ground truth. The evaluation should employ at least two structure-based graph distance measures of your choice, with a brief justification for the selected measures.

In [ ]:
# TODO - tutaj tylko wytłumaczeni wniosków bo poprzednoi to jest metodologia

## Part 2

### 1. Choose validated Boolean network
#### Task
our task is to consider a validated Boolean network model of a real-life biological mechanism. To this end, select a Boolean network 2 model of your choice from the ‘models’ subfolder of the Biodivine repository, available at https://github.com/sybila/biodivine-boolean-models, with the number of nodes (variables) not exceeding 16.‡. 

In [ ]:
# TODO - wybrac sieć

# TODO - 

### 2. Generate dataset,
#### Task
Using the insights gained from the first part of the project,
generate an appropriate dataset for the network inference task. 




In [ ]:
# TODO - tworzymy dataset (na bazie wniosków z poprzedniego)
# TODO - tutaj bierzemy wszystkie datasety, jakie wnioski udało nam się ustalić do nich

### 3. Reconstruct the network with BNFinder2 
#### Task
Reconstruct the network structure with BNFinder2, applying a scoring function chosen based on your previous
experience. Evaluate the accuracy of the reconstruction.

In [ ]:
# TODO zliczenie jakości dopasowania

# Notes

Idea behind structure is:

# Report

## Part I
## Part II

# Bibliografy

promt do klasy

będziemy programować teraz moduł kodu, gdzie wykonamy poniższą metodologie, wszystkie funkcje potrzebne do analizy mamy, moduł będzie  się skupiał tylko na opakowaniu eksperymentu, to jest.

zrobił bym taką klase która:
- przy init brała by argumenty dotyczące eksperymentu, (musiała by brać wielkości próby itd.), po czym tworzyła by z tego df'a ze wszystki warunkami eksperymentalnymi, (tak żeby można go obejrzeć)
- metoda pokazująca df'a
- metoda pokazująca ścieżke zapisu
- metoda uruchamiająca robienie testu (WAŻNE, można by ustawić równoległośc, że każdy set z jest brany po kolei, ze 3 na raz mógłbym u seibie puścić, ponieważ to są osobne procesy, i nie współdzielą pliku (wykonują funkcje z określonymi parametrami) to mogło by to znacznie przyspieszyć.

co to parametrów, konieczne jest:
- data_path: to będzie ścieżka do danych 
- experiment_name: to będzie nazwa eksperymenty, trzeba podać str
- potem trzeba dać listy (iterable, na ten moment mam taki pomysł) z parametrami, np n_samples, n_trajectories etc. i

to ostatecznie ma robić df'a z warunkami eksperymentalnymi, które będą  wyknywane odpowiednią metodą 

miejsce zapisu danych to będzie 
data_path/experiment_name/{bn_ground_truth, datasets, datasets_bnfinder, results}

to jest moja metodologia (żebyś wiedział co będzie konieczne do wstawienia w metodach żęby dało sie to wykonać)

załączam także pliki 

#### Goal
The goal of this study is to determine how the type and amount of time-series data generated from Boolean network dynamics affect the accuracy of dynamic Bayesian network (DBN) structure inference. In particular, we aim to identify:
- how the presence of attractor states in trajectories influences reconstruction accuracy,
- how trajectory length, sampling frequency, and the number of trajectories affects model metrics
- how these effects depend on network size and update dynamics,
- which data-generation techniques yield stable and informative reconstructions. 

Because synchronous and asynchronous Boolean network updates correspond to different classes of stochastic processes, all results are analyzed separately for these two update modes. Furthermore, since different scoring functions (MDL and BDe) are not directly comparable in scale, they are treated as different experiment groups for same boolean networks. 

#### Methodology

##### Relation Between Trajectory Length and Entering Attractors

**Objective.**
To characterize how trajectory length is related to the probability of entering attractors as a function of network size and dynamics.

**Experimental design.**

- The target attractor proportion is **not controlled**; trajectories evolve naturally.
- Trajectory lengths are varied in increments proportional to network size:
    - from 5 steps to 50 by 5
    - from 50 steps to 200 by 10
- Networks are grouped by size (from 4 to 16 nodes, in steps of two).
- The number of parents per node is randomly chosen from set of $\{1,2,3\}$ to avoid conditioning results on a fixed connectivity pattern.

**Measured quantities.**

- Probability of reaching an attractor as a function of trajectory length.
- How different groups (below TODO - inner link) differ in in this probability.

**Rationale.**
Attractor entry is an emergent property of the dynamics. Controlling it directly is undesirable, as it would introduce selection bias. This experiment instead characterizes the **natural scaling behavior** of Boolean network dynamics.

---

##### Impact of Sampling Frequency

**Objective.**
To determine how temporal subsampling affects autocorrelation, effective sample size, and reconstruction accuracy.

Dynamic Bayesian network inference assumes conditional independence of observations given parent states in the previous time slice. Excessive temporal dependence violates this assumption in practice by introducing redundant observations.

**Experimental design.**

- For fixed networks and trajectory lengths, datasets are generated using multiple sampling frequencies (1, 2, 3, 4, 5).
- For each dataset
    - ACF and ESS are computed,
    - MDL and BDe scores are extracted from BNFinder2 logs,

**Analysis.**
Accuracy is analyzed jointly as a function of: 
<!-- What does it mean jointly ?? -->

* sampling frequency,
* ESS,
* scoring function (MDL or BDe).

**Rationale.**
This experiment identifies sampling regimes that balance reduced autocorrelation against loss of dynamic information due to over-subsampling.

---

##### Amount of Trajectories Required for Stable Inference

**Objective.**
To determine how many independent trajectories are required to obtain statistically stable reconstructions.

**Experimental design.**

- Sampling frequency and trajectory length are fixed to values identified as near-optimal in previous experiments.
- The number of trajectories per dataset is gradually increased - from 10 to 100 by 10.
- For each setting, multiple (30) independent repetitions are performed to obtain convergent distribution .

**Evaluation.**

- Reconstruction accuracy is summarized using distributions (score functions).
- Stability is assessed by observing convergence of accuracy metrics as the number of trajectories increases.
- No classical parametric hypothesis test is assumed; instead, convergence trends is reported.
<!-- Nie znalazłem żadnego sensownego -->

**Rationale.**
Due to the randomness of Boolean functions and initial states, averaging over multiple networks is necessary to separate systematic effects from instance-specific variability.

---

#### Sets

To ensure reproducibility and comparisons, experiments are organized into predefined sets.
##### Groups
1. **Network size groups**
   Networks are grouped by number of nodes (5–16) to analyze scaling behavior.
2. **Update mode groups**
   Synchronous and asynchronous updates are treated separately, as they correspond to different stochastic processes (deterministic map vs stochastic transition system).
3. **Scoring function groups**
   MDL and BDe are analyzed independently. Absolute score values are not compared across scoring functions; only trends with respect to accuracy are considered.
4. **Random function sets**
   For each experimental condition, multiple independently generated Boolean networks are used.
   Random seeds are fixed per network/experiment (TODO ?) instance so that different data-generation parameters (sampling frequency, trajectory length, number of trajectories) are evaluated on identical underlying networks.
   <!-- is not possible we just need to use seed to experiments are reproducle -->

---

##### Averaging and Distributions

All reported results are based on **distributions**, not single values.
Depending on the experiment, averaging is performed over:

- trajectories (within a dataset),
- independent datasets,
- independently generated networks.

The aggregation strategy is explicitly chosen for each experiment to match the source of variability under investigation.

w raporcie są funkcje które poprawnie wywołują cały kod, zamieszcam je poniżej:

# Create randomo BN 
bn_random = BN(num_nodes=9, mode="asynchronous", trajectory_length=50, n_parents_per_node=[2, 3])
# save ground truth for 
bn_random.save_ground_truth(BN_TRUE_PATH / 'condition_2.csv')
# saving datasets
save_trajectories_to_csv(bn_instance=bn_random, num_trajectories=50, output_file= DATASET_PATH / 'condition_2.csv' )

# running training (uwaga tutaj trzeba te ścieżk izmienić bo tutaj jest uproszcznoe, ten metric_file powinien zalezeć od eksperymentu i powinien być łączony:
from sad2_final_project.bnfinder import run_bnfinder

datasets = [DATASET_PATH / 'condition_1.csv', DATASET_PATH / 'condition_2.csv']
datasets = [DATASET_PATH / 'condition_2.csv']
# IDEA - conditions 
for d in datasets:
    model_name = str(d.stem)
    run_bnfinder(
        #TODO -- tutaj trzeba dopasować odpowienio patha'a do próby, żeby bezmyślnei tego nei skopiować|
        dataset_path=d,
        ground_truth_path=BN_TRUE_PATH / (model_name + '.csv'),
        bnf_file_path=BN_DATASET_BNFINDER_FORMAT_PATH / (model_name + '.txt'),
        trained_model_name=BN_RESULT_PATH / model_name,
        metrics_file=BN_RESULT_PATH / (model_name + '_metric.csv'), 


Czego chce:
- napisz mi kod z taką instrukcją jak powyżej (czytelnie) i fragmentami, w przypadku używania tych funkcji, używaj ich bezpośrednio, później wpisze poprawne importy, (reszte biblitoek niezwiazanych z moimi skryptami wpisz normalnie)

